In [ ]:
import numpy as np # linear algebra
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Average, Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout, Activation, Input, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import efficientnet_v2, VGG16, ResNet50,InceptionResNetV2, InceptionV3

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

In [ ]:
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
train_directory = "drive/MyDrive/Food_Grading2/train"
val_directory = "drive/MyDrive/Food_Grading2/val"
test_directory = "drive/MyDrive/Food_Grading2/test"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=15, width_shift_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
train_directory,
target_size=(256,256),
color_mode='rgb',
batch_size=32,
class_mode='binary',
subset='training',
shuffle=True,
seed=42
)

Found 4804 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
test_directory,
target_size=(256,256),
color_mode='rgb',
batch_size=32,
class_mode='binary',
shuffle=False
)

Found 1031 images belonging to 2 classes.


In [ ]:
val_generator = test_datagen.flow_from_directory(
val_directory,
target_size=(256,256),
color_mode='rgb',
batch_size=32,
class_mode='binary',
shuffle=False
)

Found 1030 images belonging to 2 classes.


In [ ]:
input_shape = (256, 256, 3) #Cifar10 image size
resized_shape = (224, 224, 3) #EfficientNetV2B0 model input shape
num_classes = 1

def build_Inception():
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, resized_shape[:2]))(inputs) #Resize image to  size 224x224
    base_model = InceptionV3(include_top=False, input_shape=resized_shape, weights="imagenet")
    base_model.trainable = False
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    #model.trainable = False
    return model
def buildVGG():
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, resized_shape[:2]))(inputs) #Resize image to  size 224x224
    base_model = VGG16(include_top=False, input_shape=resized_shape, weights="imagenet")
    base_model.trainable = False
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    #model.trainable = False
    return model

def buildEfficientnetV2S():
    inputs = Input(shape=input_shape)
    x = tf.keras.layers.Lambda(lambda image: tf.image.resize(image, resized_shape[:2]))(inputs) #Resize image to  size 224x224
    base_model = efficientnet_v2.EfficientNetV2S(include_top=False, input_shape=resized_shape, weights="imagenet")
    base_model.trainable = False
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    #model.trainable = False
    return model

In [ ]:
inception = build_Inception()
efficientnet = buildEfficientnetV2S()
VGG16 = buildVGG()

In [ ]:
inception.load_weights('/content/drive/MyDrive/Inceptionv3_Model.keras')
efficientnet.load_weights('/content/drive/MyDrive/Efficientnet_V2S_Model.keras')
VGG16.load_weights('/content/drive/MyDrive/VGGFinal.keras')

In [ ]:
inception.trainable = False
efficientnet.trainable = False
VGG16.trainable = False

In [ ]:
input_shape = (256, 256, 3) #Cifar10 image size
resized_shape = (224, 224, 3) #EfficientNetV2B0 model input shape


In [ ]:
models = [inception, efficientnet, VGG16]
model_input = Input(shape=input_shape)
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
model = Model(inputs=model_input, outputs=ensemble_output)

In [ ]:


plateau = ReduceLROnPlateau(monitor="val_loss", factor=0.7, patience=1, verbose=1)
earlystopping = EarlyStopping(monitor="val_loss", patience=3, verbose=1)

model.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9),
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()
print("\n")




Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 1)              │     21,804,833 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_1 (Functional) │ (None, 1)              │     20,332,641 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_2 (Functional) │ (None, 1)              │     14,715,201 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average (Average)         │ (None, 1)              │              0 │ functional[0][0],      │
│                           │                        │                │ functional_1[0][0],    │
│                           │                        │                │ functional_2[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 56,852,675 (216.88 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 56,852,675 (216.88 MB)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

Evaluate on test data


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 465s 14s/step - accuracy: 0.9822 - loss: 0.2606
test loss, test acc: [0.28463831543922424, 0.9767216444015503]
